In [43]:
import json

with open("clickup_space_schema.json") as f:
    schema = json.load(f)
    
schema

[{'name': 'get_spaces',
  'description': 'View the Spaces available in a Workspace.',
  'parameters': {'type': 'object',
   'properties': {'team_id': {'type': 'string',
     'description': 'The ID of the team'},
    'archived': {'type': 'boolean',
     'description': 'A flag to decide whether to include archived spaces or not'}},
   'required': ['team_id', 'archived']}},
 {'name': 'create_space',
  'description': 'Add a new Space to a Workspace.',
  'parameters': {'type': 'object',
   'properties': {'team_id': {'type': 'string',
     'description': 'The ID of the team'},
    'name': {'type': 'string', 'description': 'The name of the new space'},
    'multiple_assignees': {'type': 'boolean',
     'description': 'Enable or disable multiple assignees for tasks within the space'},
    'features': {'type': 'object',
     'description': 'Enabled features within the space',
     'properties': {'due_dates': {'type': 'object',
       'description': 'Due dates feature settings',
       'properti

In [44]:
def get_key_trees(schema_props):
    # print(schema_props)
    key_list = []
    for k, v in schema_props.items():
        if v["type"] != "object":
            key_list.append(f'{k}:{v["type"]}')
        else:
            branch = get_key_trees(v["properties"])
            key_list.append(f'{k}:{branch}')
    return f'({", ".join(key_list)})'
            
    
for sc in schema:
    print(f'{sc["name"]}{get_key_trees(sc["parameters"]["properties"])}')
    # print()


get_spaces(team_id:string, archived:boolean)
create_space(team_id:string, name:string, multiple_assignees:boolean, features:(due_dates:(enabled:boolean, start_date:boolean, remap_due_dates:boolean, remap_closed_due_date:boolean), time_tracking:(enabled:boolean)))
get_space(space_id:string)
update_space(space_id:string, name:string, color:string, private:boolean, admin_can_manage:boolean, multiple_assignees:boolean, features:(due_dates:(enabled:boolean, start_date:boolean, remap_due_dates:boolean, remap_closed_due_date:boolean), time_tracking:(enabled:boolean)))
delete_space(space_id:string)
get_space_tags(space_id:string)
create_space_tag(space_id:string, tag:(name:string, tag_fg:string, tag_bg:string))
delete_space_tag(space_id:string, tag_name:string, tag:(name:string, tag_fg:string, tag_bg:string))


In [45]:
import os
import json
import dotenv
from textwrap import dedent
from datetime import datetime

dotenv.load_dotenv()

tools = [{"name":sc["name"],  "description":sc["description"], "input_schema": sc["parameters"]} for sc in schema]
tools

[{'name': 'get_spaces',
  'description': 'View the Spaces available in a Workspace.',
  'input_schema': {'type': 'object',
   'properties': {'team_id': {'type': 'string',
     'description': 'The ID of the team'},
    'archived': {'type': 'boolean',
     'description': 'A flag to decide whether to include archived spaces or not'}},
   'required': ['team_id', 'archived']}},
 {'name': 'create_space',
  'description': 'Add a new Space to a Workspace.',
  'input_schema': {'type': 'object',
   'properties': {'team_id': {'type': 'string',
     'description': 'The ID of the team'},
    'name': {'type': 'string', 'description': 'The name of the new space'},
    'multiple_assignees': {'type': 'boolean',
     'description': 'Enable or disable multiple assignees for tasks within the space'},
    'features': {'type': 'object',
     'description': 'Enabled features within the space',
     'properties': {'due_dates': {'type': 'object',
       'description': 'Due dates feature settings',
       'prop

In [46]:
import anthropic
client = anthropic.Anthropic()

response = client.beta.tools.messages.create(
  model="claude-3-haiku-20240307",
  messages=[

    {
      "role": "user",
      "content": """\
The software development team at FastTech recently concluded a project, and it’s been 
decided to archive the specific ClickUp space identified by \"sd456\" to ensure it doesn't 
clutter the current workspace. Before doing so, the team leader wants to retrieve the list 
of custom tags created in this space for reporting purposes."""
    },
  ],

  temperature=1,
  max_tokens=4096,
  top_p=1,
  tools=tools,
)

response

ToolsBetaMessage(id='msg_01KRvmvLuAh73jk1gZ7g9xiG', content=[TextBlock(text='Okay, let\'s retrieve the list of custom tags created in the ClickUp space with the ID "sd456":', type='text'), ToolUseBlock(id='toolu_018mPddMt2uXaGr9iv83mEwY', input={'space_id': 'sd456'}, name='get_space_tags', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=1656, output_tokens=84))

In [47]:
import json

with open("clickup_space_benchmark.json") as f:
    bench_data = json.load(f)
    
bench_data[1]

{'prompt': 'Ella, the project coordinator, is setting up a new project space in ClickUp for the "Creative Minds" team with team ID "cm789". This space, named "Innovative Campaigns 2023", should allow multiple assignees for tasks, but keep due dates and time tracking disabled, as the initial planning phase doesn\'t require strict deadlines or time monitoring.',
 'solution': 'create_space(team_id="cm789", name="Innovative Campaigns 2023", multiple_assignees=True, features=(due_dates=(enabled=False, start_date=False, remap_due_dates=False, remap_closed_due_date=False), time_tracking=(enabled=False)))'}

In [48]:
bench_data[1]["solution"]

'create_space(team_id="cm789", name="Innovative Campaigns 2023", multiple_assignees=True, features=(due_dates=(enabled=False, start_date=False, remap_due_dates=False, remap_closed_due_date=False), time_tracking=(enabled=False)))'

In [49]:
for b in bench_data:
    print(b["solution"])

get_spaces(team_id="team123", archived=False)
create_space(team_id="cm789", name="Innovative Campaigns 2023", multiple_assignees=True, features=(due_dates=(enabled=False, start_date=False, remap_due_dates=False, remap_closed_due_date=False), time_tracking=(enabled=False)))
get_space_tags(space_id="sd456")
create_space_tag(space_id="prj1122", tag=(name="Critical Bug", tag_fg="#FFFFFF", tag_bg="#8B0000"))
delete_space_tag(space_id="qa789", tag_name="MinorIssue", tag=(name="MinorIssue", tag_fg="#000000", tag_bg="#FFFFE0"))
update_space(space_id="bkend345", name="Backend_Development_Space", color="#000000", private=False, admin_can_manage=True, multiple_assignees=False, features=(due_dates=(enabled=True, start_date=True, remap_due_dates=True, remap_closed_due_date=False), time_tracking=(enabled=True)))
delete_space(space_id="gt678")
create_space(team_id="ds101", name="Design Studio 2023", multiple_assignees=True, features=(due_dates=(enabled=True, start_date=True, remap_due_dates=False, re

In [50]:
import anthropic
client = anthropic.Anthropic()

fcalling_llm = lambda fprompt : client.beta.tools.messages.create(
  model="claude-3-haiku-20240307",
  messages=[
    {
      "role": "user",
      "content": fprompt
    },
  ],
  temperature=0,
  max_tokens=4096,
  top_p=1,
  tools=tools,
)

response = fcalling_llm(bench_data[1]["prompt"])
# response
response

ToolsBetaMessage(id='msg_01MAGVfKhoxSjqkabqz6jFTk', content=[TextBlock(text='Okay, let\'s create the new space for the "Creative Minds" team with the desired configurations:', type='text'), ToolUseBlock(id='toolu_017FUhGAoEXSmxhVj7GLM1Ka', input={'team_id': 'cm789', 'name': 'Innovative Campaigns 2023', 'multiple_assignees': True, 'features': {'due_dates': {'enabled': False, 'remap_closed_due_date': False, 'remap_due_dates': False, 'start_date': False}, 'time_tracking': {'enabled': False}}}, name='create_space', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=1662, output_tokens=212))

In [51]:
import re 
from pprint import pprint

class FuncStringParser():
    def __init__(self):
        self.record_keeper = {}
        
    def replace_with_dict(self, match):
        replacement = f"dict{len(self.record_keeper)}"
        self.record_keeper[replacement] = match.group()
        # dict_counter += 1
        return replacement

    def arg_tuple_str_to_args_dict(self, func_args):
        # print(func_args)
        args_dict = {}
        func_args = func_args[1:-1].split(",")
        for arg in func_args:
            k, v = arg[:arg.find("=")], arg[arg.find("=")+1:]
            k, v = k.strip(), v.strip()
            
            if v.startswith("(") and v.endswith(")"):
                v = self.func_to_args_dict(v)
                # print(v)
            
            args_dict[k] = v

        return args_dict

    def replace_true_false_dfs(self, nested_dict):
        for key, value in nested_dict.items():
            # If the value is a nested dictionary, recursively process it
            if isinstance(value, dict):
                self.replace_true_false_dfs(value)
            else:
                nested_dict[key] = eval(value)
                
                    
    def func_to_args_dict(self, func_args):
        pattern_uncompiled = r'\(([^()]+)\)'
        pattern = re.compile(pattern_uncompiled)
        # print(re.findall(pattern_uncompiled, func_args))
        while len(re.findall(pattern_uncompiled, func_args)) > 1:
            func_args = re.sub(pattern, self.replace_with_dict, func_args)
            
        # print(re.findall(pattern_uncompiled, func_args))
        if sum([m.start(0) > 2 for m in re.finditer(r'\(([^()]+)\)', func_args)]):
            func_args = re.sub(pattern, self.replace_with_dict, func_args)
            # print(func_args)
        func_args = re.sub(pattern, self.replace_with_dict, func_args)
        # pprint(func_args)
            
        dict_record_keeper = {k:self.arg_tuple_str_to_args_dict(v) for k, v in self.record_keeper.items()}
        # print(dict_record_keeper)
        
        # inv_dict_record_keeper = {v:k for k,v in dict_record_keeper.items()}
        for i1, (k1, v1) in enumerate(dict_record_keeper.items()):
            for i2, (k2, v2) in enumerate(dict_record_keeper.items()):
                if i2 == i1:
                    break
                if k2 in v1.values():
                    # print(k2, v1)
                    dict_record_keeper[k1][list(v1.keys())[list(v1.values()).index(k2)]] = v2
        args_dict = list(dict_record_keeper.values())[-1]
        self.replace_true_false_dfs(args_dict)
        self.record_keeper = {}
        return args_dict
    
    def func_string_to_func_dict(self, func_string):
        func_name = func_string[:func_string.find("(")]
        func_args = func_string[func_string.find("("):]
        func_args_dict = self.func_to_args_dict(func_args)
        return {"name": func_name, "arguments": func_args_dict}
    
    def llm_response_to_func_dict(self, llm_response):
        for message in llm_response.content:
            if message.type == "tool_use":
                return {
                    "name": message.name,
                    "arguments": message.input,
                }
        return {"name": None, "arguments": None}
        

                    
        
    
# Sample string
func = 'create_space(team_id="cm789", name="Innovative Campaigns 2023", multiple_assignees=True, features=(due_dates=(enabled=False, start_date=False, remap_due_dates=False, remap_closed_due_date=False), time_tracking=(enabled=False)))'

func_name = func[:func.find("(")]
func_args = func[func.find("("):]

fsp = FuncStringParser()
# fsp.func_to_args_dict(func_args)

fsp.func_string_to_func_dict(func)


{'name': 'create_space',
 'arguments': {'team_id': 'cm789',
  'name': 'Innovative Campaigns 2023',
  'multiple_assignees': True,
  'features': {'due_dates': {'enabled': False,
    'start_date': False,
    'remap_due_dates': False,
    'remap_closed_due_date': False},
   'time_tracking': {'enabled': False}}}}

In [52]:
{"space_id":"sd456"} == {"space_id":"sd456"}

True

In [53]:
from tqdm.auto import tqdm

llm_output = []

for bdata in tqdm(bench_data):
    response = fcalling_llm(bdata["prompt"])
    try:
        llm_output.append(response)
    except TypeError:
        print(bdata["prompt"])
        print(response)
        llm_output.append(None)
        print("------")
llm_output

100%|██████████| 50/50 [01:28<00:00,  1.78s/it]


[ToolsBetaMessage(id='msg_013A5szQvh9Ro46cqgWLSNCQ', content=[TextBlock(text='Okay, let\'s use the relevant tools to check the active spaces under the "Innovative Solutions" team.', type='text'), ToolUseBlock(id='toolu_01EieNPhib7dUgXuRrW4eYEN', input={'team_id': 'team123', 'archived': False}, name='get_spaces', type='tool_use')], model='claude-3-haiku-20240307', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=1656, output_tokens=97)),
 ToolsBetaMessage(id='msg_01XLE6NsYnWqse7RJGN8g31P', content=[TextBlock(text='Okay, let\'s create the new space for the "Creative Minds" team with the desired configurations:', type='text'), ToolUseBlock(id='toolu_01XUD1oMuMWmfmWXXJ3Jae2w', input={'team_id': 'cm789', 'name': 'Innovative Campaigns 2023', 'multiple_assignees': True, 'features': {'due_dates': {'enabled': False, 'remap_closed_due_date': False, 'remap_due_dates': False, 'start_date': False}, 'time_tracking': {'enabled': False}}}, name='cr

In [54]:
from pprint import pprint

fsp = FuncStringParser()
result_list = []

# fsp.func_to_args_dict(func_args)
for bdata, llm_response in zip(bench_data, llm_output):
    bdata_solution_dict = fsp.func_string_to_func_dict(bdata["solution"])
    llm_response_dict = fsp.llm_response_to_func_dict(llm_response)
    
    pprint(bdata_solution_dict)
    pprint(llm_response_dict)
    print(llm_response_dict == bdata_solution_dict)
    result_list.append(llm_response_dict == bdata_solution_dict)
    print("-----")

{'arguments': {'archived': False, 'team_id': 'team123'}, 'name': 'get_spaces'}
{'arguments': {'archived': False, 'team_id': 'team123'}, 'name': 'get_spaces'}
True
-----
{'arguments': {'features': {'due_dates': {'enabled': False,
                                          'remap_closed_due_date': False,
                                          'remap_due_dates': False,
                                          'start_date': False},
                            'time_tracking': {'enabled': False}},
               'multiple_assignees': True,
               'name': 'Innovative Campaigns 2023',
               'team_id': 'cm789'},
 'name': 'create_space'}
{'arguments': {'features': {'due_dates': {'enabled': False,
                                          'remap_closed_due_date': False,
                                          'remap_due_dates': False,
                                          'start_date': False},
                            'time_tracking': {'enabled': False}},
           

In [55]:
run_acc_1 = sum(result_list)/len(result_list)
run_acc_1

0.48

In [56]:
from pprint import pprint
from tqdm.auto import tqdm

llm_output = []

for bdata in tqdm(bench_data):
    response = fcalling_llm(bdata["prompt"])
    try:
        llm_output.append(response)
    except TypeError:
        print(bdata["prompt"])
        print(response)
        print("-------")
        llm_output.append(None)

100%|██████████| 50/50 [01:29<00:00,  1.79s/it]


In [57]:
from pprint import pprint

fsp = FuncStringParser()
result_list = []

# fsp.func_to_args_dict(func_args)
for bdata, llm_response in zip(bench_data, llm_output):
    bdata_solution_dict = fsp.func_string_to_func_dict(bdata["solution"])
    llm_response_dict = fsp.llm_response_to_func_dict(llm_response)
    
    pprint(bdata_solution_dict)
    pprint(llm_response_dict)
    print(llm_response_dict == bdata_solution_dict)
    result_list.append(llm_response_dict == bdata_solution_dict)
    print("-----")
    
run_acc_2 = sum(result_list)/len(result_list)
run_acc_2

{'arguments': {'archived': False, 'team_id': 'team123'}, 'name': 'get_spaces'}
{'arguments': {'archived': False, 'team_id': 'team123'}, 'name': 'get_spaces'}
True
-----
{'arguments': {'features': {'due_dates': {'enabled': False,
                                          'remap_closed_due_date': False,
                                          'remap_due_dates': False,
                                          'start_date': False},
                            'time_tracking': {'enabled': False}},
               'multiple_assignees': True,
               'name': 'Innovative Campaigns 2023',
               'team_id': 'cm789'},
 'name': 'create_space'}
{'arguments': {'features': {'due_dates': {'enabled': False,
                                          'remap_closed_due_date': False,
                                          'remap_due_dates': False,
                                          'start_date': False},
                            'time_tracking': {'enabled': False}},
           

0.48

In [58]:
from pprint import pprint
from tqdm.auto import tqdm

llm_output = []

for bdata in tqdm(bench_data):
    response = fcalling_llm(bdata["prompt"])
    try:
        llm_output.append(response)
    except TypeError:
        print(bdata["prompt"])
        print(response)
        print("-------")
        llm_output.append(None)

100%|██████████| 50/50 [01:27<00:00,  1.76s/it]


In [59]:
from pprint import pprint

fsp = FuncStringParser()
result_list = []

# fsp.func_to_args_dict(func_args)
for bdata, llm_response in zip(bench_data, llm_output):
    bdata_solution_dict = fsp.func_string_to_func_dict(bdata["solution"])
    llm_response_dict = fsp.llm_response_to_func_dict(llm_response)
    
    pprint(bdata_solution_dict)
    pprint(llm_response_dict)
    print(llm_response_dict == bdata_solution_dict)
    result_list.append(llm_response_dict == bdata_solution_dict)
    print("-----")
    
run_acc_3 = sum(result_list)/len(result_list)
run_acc_3

{'arguments': {'archived': False, 'team_id': 'team123'}, 'name': 'get_spaces'}
{'arguments': {'archived': False, 'team_id': 'team123'}, 'name': 'get_spaces'}
True
-----
{'arguments': {'features': {'due_dates': {'enabled': False,
                                          'remap_closed_due_date': False,
                                          'remap_due_dates': False,
                                          'start_date': False},
                            'time_tracking': {'enabled': False}},
               'multiple_assignees': True,
               'name': 'Innovative Campaigns 2023',
               'team_id': 'cm789'},
 'name': 'create_space'}
{'arguments': {'features': {'due_dates': {'enabled': False,
                                          'remap_closed_due_date': False,
                                          'remap_due_dates': False,
                                          'start_date': False},
                            'time_tracking': {'enabled': False}},
           

0.48

In [60]:
print("ClickUp Benchmark - No System Prompt : Accuracy")
print("=========================================")
print(f"Run 1: {run_acc_1}")
print(f"Run 2: {run_acc_2}")
print(f"Run 3: {run_acc_3}")


ClickUp Benchmark - No System Prompt : Accuracy
Run 1: 0.48
Run 2: 0.48
Run 3: 0.48
